<a name="learn4-top"></a> 
# Bloomberg Python Series: <br><span style="color:orange"> Using the Bloomberg Query Language with Fixed Income Data II </span>

### Topics
1. [__Recap__](#learn4-topic1)
- [1.1 Peer Curve Analysis](#learn4-topic1.1)
- [1.2 Index Analysis](#learn4-topic1.2)
2. [__Tools for Cross-Asset Analysis__](#learn4-topic2)
- [2.1 Nested Filters](#learn4-topic2.1)
- [2.2 Data Availability Management](#learn4-topic2.2)
- [2.3 Grouping II - Ungroup](#learn4-topic2.3)
- [2.4 Cut](#learn4-topic2.4)
- [2.5 Relative Evaluation (Groupxyz)](#learn4-topic2.5)
- [2.6 Identifying Outliers](#learn4-topic2.6)
- [2.7 Issuer](#learn4-topic2.7)
- [2.8 Value](#learn4-topic2.8)
3. [__Summary__](#learn4-topic3)

<a id='learn4-topic1'></a>

<span style="color:darkorange; font-size:2em"> 1 Recap </span>

Let's import our packages and initiate the connection to BQL first

In [1]:
import bql
import pandas as pd
from collections import OrderedDict

In [2]:
bq = bql.Service()

### 1.1 Peer Curve Analysis

What was the median OAS spread by year of maturity for the bonds of Novartis (NOVN SW), Roche (ROG SW) and GSK (GSK LN)?

In [3]:
#Find bonds
bonds = bq.univ.bonds(['NOVN SW Equity','ROG SW Equity','GSK LN Equity'])#Retrieve median OAS Spread             
yld = bq.data.yield_()
mat = bq.data.maturity().year()
yield_by_year = yld.group(mat).median()

req = bql.Request(bonds, {'Median Yield by Year':yield_by_year})
res = bq.execute(req)
data = res[0].df()
data

,DATE,ORIG_IDS,YEAR(MATURITY()),Median Yield by Year
ID,,,,
2021.0,2021-04-08,None,2021,-0.393240
2022.0,2021-04-08,None,2022,0.268339
2023.0,2021-04-08,None,2023,-0.241071
2024.0,2021-04-08,None,2024,0.191204
2025.0,2021-04-08,None,2025,-0.121881
2026.0,2021-04-08,None,2026,-0.064422
2027.0,2021-04-08,None,2027,1.379508
2028.0,2021-04-08,BJ258063 Corp,2028,1.350976
2029.0,2021-04-08,None,2029,0.081202


In [4]:
import bql
bq = bql.Service()

bql_item = bq.func.avg(bq.func.group(bq.data.spread(), bq.data.classification_name()))

bql_universe = bq.univ.members('LEGATRUU Index')

bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)
data = bql_response[0].df()
data

,DATE,ORIG_IDS,CLASSIFICATION_NAME(),"AVG(GROUP(SPREAD(),CLASSIFICATION_NAME()))"
ID,,,,
,NaT,None,,NaN
Communications,2021-04-08,None,Communications,90.029407
Consumer Discretionary,2021-04-08,None,Consumer Discretionary,75.899700
Consumer Staples,2021-04-08,None,Consumer Staples,70.340771
Energy,2021-04-08,None,Energy,117.616881
Financials,2021-04-08,None,Financials,59.197125
Government,2021-04-08,None,Government,18.817207
Health Care,2021-04-08,None,Health Care,76.583975
Industrials,2021-04-08,None,Industrials,84.100908


### 1.2 Index Analysis

What was the average change in yield in the last 7 days for bonds in the Bloomberg Barclays China Treasury + Policy Bank Total Return Index CNY (I32561CN Index)?

In [5]:
#Define index
index = bq.univ.members('I32561CN Index')
#Yield Change
yld = bq.data.yield_(dates=bq.func.range('-7d','0d'), fill='prev')
yld_chg = yld.net_chg()
#Average change across the index
avg_yld_chg = yld_chg.group().avg()

req = bql.Request(index, {'Avg Change in Yield':avg_yld_chg})
res = bq.execute(req)
data = res[0].df()
data

,DATE,ORIG_IDS,Avg Change in Yield
ID,,,
IdentityGroup,2021-04-08,None,0.00137


### 1.3 New Issuance Analysis

What was the breakdown of active (ie not withdrawn or amended) Syndicated(ie institutional) Loans issued in EUR by Year and Month of Issue since 2018? (Hint: Use Loan Tranche Size and Launch dates)

In [6]:
#loans universe
loans = bq.univ.loansuniv('All')
#Let's construct our filters
filter_syndicated = bq.data.institutional_loan() == 'TRUE'
filter_active1 = bq.data.ln_issue_status() != 'Withdrawn'
filter_active2 = bq.data.ln_issue_status() != 'Cancelled'
filter_active = bq.func.and_(filter_active1, filter_active2)
filter_crncy = bq.data.crncy() == 'EUR'
filter_date = bq.data.launch_date() >= '2018-01-01'
#Combine our filters and apply
filters = bq.func.and_(filter_date, bq.func.and_(filter_crncy,bq.func.and_(filter_syndicated, filter_active)))
filtered_loans = bq.univ.filter(loans, filters)
#Get loan tranche size by year and month
loan_size = bq.data.ln_tranche_size() / 10 ** 6
launch_date = bq.data.launch_date()
launch_date = launch_date.year() * 100 + launch_date.month()
loan_size_by_date = loan_size.group(launch_date).sum()

req = bql.Request(filtered_loans, {'Issuance': loan_size_by_date})
res = bq.execute(req)
data = res[0].df()
data


,CURRENCY,ORIG_IDS,(YEAR(LAUNCH_DATE())*100)+MONTH(LAUNCH_DATE()),Issuance
ID,,,,
201801.0,None,None,201801.0,17020.265000
201802.0,None,None,201802.0,7585.022500
201803.0,None,None,201803.0,7114.489463
201804.0,None,None,201804.0,6161.343891
201805.0,None,None,201805.0,12301.060000
201806.0,None,None,201806.0,9099.305519
201807.0,None,None,201807.0,7085.800000
201808.0,None,None,201808.0,5455.000000
201809.0,None,None,201809.0,5287.753007


[&uarr; Return to Top](#learn4-top)

<a id='learn4-topic2'></a>

<span style="color:darkorange; font-size:2em"> 2 Tools for Cross-Asset Analysis </span>



<img src="../../Visualisations/BQL Connections.jpg" style="width: 600px;"/>

<a id='learn4-topic2.1'></a>

### 2.1 Nested Filters



- Multiple filters means ability to efficiently screen your universes on BQL on multiple layers
- One application is issuer analysis (from equity to bond)

<img src="../../Visualisations/BQL Nested Filter.jpg" style="width: 600px;"/>

Q: Screen for the fixed coupon bonds with a maturity greater than 10 years for members of S&P500 with a high probability of default (net debt to Ebitda> 3) and find the average yield by Moody's credit rating


In [ ]:
#Let's screen our Equities
eq_index = bq.univ.members('SPX Index')
filter_lev = bq.data.net_debt_to_ebitda(FPT='A') >= 3
filter_sector = bq.data.classification_name('BICS','1') != 'Financials'
criteria = bq.func.and_(filter_lev, filter_sector)
#apply our screening criteria to our Equity Index
filtered_eq_index = bq.univ.filter(eq_index, criteria)
#Let's find the bonds for these screened issuers
bonds = bq.univ.bonds(filtered_eq_index)
#apply our bond criteria
filter_cpn = bq.data.cpn_typ() == 'FIXED'
filter_mat = bq.data.maturity() > '10Y'
bond_criteria = bq.func.and_(filter_cpn, filter_mat)
#let's find our ultimate list of securities based on our equity screen and our fixed income screen
filtered_bonds = bq.univ.filter(bonds, bond_criteria)
#For these bonds, let's get average median by Moody rating
yld = bq.data.yield_()
rating = bq.data.credit_rating(CREDIT_RATING_SOURCE='MOODY')
avg_yld = yld.group(rating).median()

req = bql.Request(filtered_bonds, {'Avg Yield by Rating':avg_yld})
res = bq.execute(req)
data = res[0].df()
data

<a id='learn4-topic2.2'></a>

### 2.2 Data availability management



- alternatives for unavailable data avail()

In [7]:
#Getting either Ebitda or Operating Income per Member of SMI
index = bq.univ.members('SMI Index')
ebitda = bq.data.ebitda(CURRENCY='USD')
op_inc = bq.data.is_oper_inc(CURRENCY='USD')
#Take Ebitda if available, otherwise Operating Income
avail_ebitda = bq.func.avail(ebitda, op_inc)
ev = bq.data.enterprise_value(CURRENCY='USD')
ebitda_yield = avail_ebitda/ev

req = bql.Request(index, {'Ebitda Yield':ebitda_yield})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Ebitda Yield
ID,,,,,
ABBN SE Equity,2021-02-26,2020-12-31,2021-04-08,None,0.035386
CFR SE Equity,2020-11-06,2020-09-30,2021-04-08,None,0.071243
CSGN SE Equity,2021-03-18,2020-12-31,2021-04-08,None,NaN
GEBN SE Equity,2021-03-10,2020-12-31,2021-04-08,None,0.039429
GIVN SE Equity,2021-01-29,2020-12-31,2021-04-08,None,0.035201
LHN SE Equity,2021-02-26,2020-12-31,2021-04-08,None,0.169304
LONN SE Equity,2021-03-25,2020-12-31,2021-04-08,None,0.031208
NESN SE Equity,2021-03-25,2020-12-31,2021-04-08,None,0.052233
NOVN SE Equity,2021-01-26,2020-12-31,2021-04-08,None,0.067508


<a id='learn4-topic2.3'></a>

### 2.3 Grouping II - Ungroup



- Bucket level calculation with output at security level
- eg what is the rank of each security's rank within its own sector?



<img src="../../Visualisations/BQL Groupxyz.jpg" style="width: 600px;"/>

<a id='learn4-topic2.4'></a>

### 2.4 Cut



- Cut() allows us to slice and dice our data in percentiles/quartiles/quintiles


In [8]:
#Find the percentile of Vodafone bonds based on ASW Spread
bonds = bq.univ.bonds('VOD LN Equity')
sprd = bq.data.spread(SPREAD_TYPE='ASW')
sprd_pctl = sprd.group().cut(100).ungroup()

req = bql.Request(bonds, {'Percentile of ASW Spread':sprd_pctl})
res = bq.execute(req)
data = res[0].df()
data

,DATE,Percentile of ASW Spread
ID,,
AS779476 Corp,2021-04-08,11.0
AX916964 Corp,2021-04-08,99.0
AS779475 Corp,2021-04-08,33.0
EJ552316 Corp,2021-04-08,21.0
JK203775 Corp,2021-04-08,9.0
EJ372541 Corp,2021-04-08,16.0
QZ698193 Corp,2021-04-08,14.0
ZO136566 Corp,2021-04-08,97.0
EI109926 Corp,2021-04-08,7.0


In [9]:
#Analyze the top 20th percentile of bonds by ASW Spread for Vodafone bonds
bonds = bq.univ.bonds('VOD LN Equity')
sprd = bq.data.spread(SPREAD_TYPE='ASW')
sprd_pctl = sprd.group().cut(100).ungroup()
criteria = sprd_pctl >= 80.0
filtered_bonds = bq.univ.filter(bonds, criteria)
name = bq.data.name().group().count()

req = bql.Request(bonds, {'Name':name})
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS,Name
ID,,
IdentityGroup,None,62


#### 2.4.1 Ungroup & Cut Practice

Show me the bottom quintile (1/5) of members of the EM USD High Yield Index ('I31732US Index') with the lowest maturity relative to today?

In [10]:
#Enter answer here
index = bq.univ.members('I31732US Index')
mat = bq.data.maturity()
today = bq.func.today()
relative_mat = (mat - today) / 365.25
rel_mat_qtl = relative_mat.group().cut(5).ungroup()
#Create the filter
filter_rel_mat = rel_mat_qtl == 1
filtered_index = bq.univ.filter(index, filter_rel_mat)

req = bql.Request(filtered_index, {'Relative Maturity':relative_mat.round(3)})
res = bq.execute(req)
data = res[0].df()
data

,Relative Maturity
ID,
AM715788 Corp,1.944
AO063629 Corp,2.218
AR725801 Corp,1.933
AX957438 Corp,0.999
BK517726 Corp,2.281
EJ419529 Corp,1.599
AS239074 Corp,2.026
EJ636932 Corp,2.045
AZ097567 Corp,1.180


<a id='learn4-topic2.5'></a>

### 2.5 Relative Valuation (Groupxyz)



- Using group().func().ungroup() can be unwieldy.
- The groupxyz functions make this easier by making the manipulation in the background.
- Common applications include grouprank(), groupzscore(), grouprank(), groupavg() and groupmedian()
- You can stack groupxyz() functions to manage your output within BQL.

In [11]:
#Assess the top 10 bonds in the US Corporate High Yield Index (1900 Members) with the greatest I-Spread widening in the last month

index = bq.univ.members('LF98TRUU Index')
#Define spread change
sprd = bq.data.spread(SPREAD_TYPE='I', dates=bq.func.range('-1m','0d'), fill='prev')
sprd_chg = sprd.net_chg()
sprd_chg_rank = sprd_chg.grouprank(order='desc')
#Filter by rank
criteria = sprd_chg_rank <= 20
filtered_index = bq.univ.filter(index, criteria)

#Get results
req = bql.Request(filtered_index, {'Top 10 Bonds to Watch':sprd_chg.groupsort()})
res = bq.execute(req)
data = res[0].df()
data

,DATE,Top 10 Bonds to Watch
ID,,
EJ460599 Corp,2021-04-08,19915.376408
ZO531871 Corp,2021-04-08,15759.882126
AZ202088 Corp,2021-04-08,5248.738708
AQ023271 Corp,2021-04-08,3968.414337
AT540923 Corp,2021-04-08,1268.206238
AP093654 Corp,2021-04-08,1049.025513
AM342846 Corp,2021-04-08,958.050293
AL501984 Corp,2021-04-08,847.110413
AP841309 Corp,2021-04-08,643.797943


In [12]:
#Show me the average coupon by year of maturity of USD green bonds issued in 2019 with a higher yield than the median of their BCLASS sector peers (BCLASS, 1)

active_bonds = bq.univ.bondsuniv('Active')
#Filter Criteria
criteria_green = bq.data.green_bond_loan_indicator() == 'True'
criteria_date = bq.data.issue_dt().year() == '2019'
criteria_crncy = bq.data.crncy() == 'USD'
#Combine our filter criteria and apply
green_criteria = bq.func.and_(criteria_crncy, bq.func.and_(criteria_green, criteria_date))
filtered_green_bonds = bq.univ.filter(active_bonds, green_criteria)
#Yield higher than sector peers
yld = bq.data.yield_()
sector = bq.data.classification_name('BCLASS','1')
criteria_yld = yld > yld.groupmedian(sector)
#Filter the green bonds
green_and_yield = bq.univ.filter(filtered_green_bonds, criteria_yld)
#Average Coupon by Maturity Year
coupon = bq.data.cpn()
mat = bq.data.maturity()
mat_year = mat.year()
avg_coupon_by_year = coupon.group(mat_year).avg()

req = bql.Request(green_and_yield, {'Avg Coupon by Year of Maturity':avg_coupon_by_year})
res = bq.execute(req)
data = res[0].df()
data

,CPN_TYP,MULTIPLIER,ORIG_IDS,YEAR(MATURITY()),Avg Coupon by Year of Maturity
ID,,,,,
2021.0,FIXED,1.0,ZS107285 Corp,2021,12.850000
2022.0,FIXED,1.0,None,2022,6.783333
2023.0,FIXED,1.0,ZQ435931 Corp,2023,5.750000
2024.0,None,1.0,None,2024,4.605069
2025.0,FIXED,1.0,AZ744309 Corp,2025,5.550000
2026.0,FIXED,1.0,AZ744286 Corp,2026,5.950000
2028.0,FIXED,1.0,ZQ911288 Corp,2028,4.750000
2029.0,FIXED,1.0,None,2029,4.019714
2030.0,FIXED,1.0,ZQ259068 Corp,2030,3.932500


#### 2.5.1 Groupxyz Practice: Falling Angels

Which were the top 20 bonds, whose spread widened the most (in %) in the past 2 weeks relative to the average of their sector peers whose spread widened, as defined by bonds in the same BCLASS Sector Level 3 in the Bloomberg Barclays APAC AAA Total Return Unhedged JPY (LP06TREU Index)?

In [ ]:
#Enter answer here
index = bq.univ.members('LP06TREU Index')
spread_chg = bq.data.spread(SPREAD_TYPE='OAS', dates=bq.func.range('-2w','0d'), fill='prev').pct_chg()
sector = bq.data.classification_name('BCLASS','3')
sector_chg = spread_chg.groupavg(sector)
relative_chg = spread_chg/sector_chg
#Create filters and apply
filter_relative = relative_chg > 0
filter_widen = spread_chg > 0
filters_movement = bq.func.and_(filter_relative, filter_widen)
filtered_bonds = bq.univ.filter(index, filters_movement)
#get the top 20
filter_relative_rank = relative_chg.grouprank() <= 20
top_20_bonds = bq.univ.filter(filtered_bonds, filter_relative_rank)
securityid = bq.data.id()

req = bql.Request(top_20_bonds, securityid)
res = bq.execute(req)
data = res[0].df()
data

<a id='learn4-topic2.6'></a>

### 2.6 Identifying outliers 



- Z-score calculations on Bloomberg Server 
- identify, locate and manage

In [3]:
#Find the top 5 outliers for the percentage spread change in bonds in the energy sector of an index in the past week
#Let's get our universe of Energy bonds
index = bq.univ.members('LBUSTRUU Index')
sector = bq.data.classification_name('BCLASS','3')
filter_sector = sector == 'Energy'
filtered_index = bq.univ.filter(index, filter_sector)
#Spread Change Z-Score
spread_chg = bq.data.spread(dates=bq.func.range('-1w','0d'), fill='prev').pct_chg()
spread_zscore = spread_chg.groupzscore()
spread_zscore_abs = spread_zscore.abs()
spread_zscore_abs = spread_zscore_abs.grouprank()
#Put in place our second filter
filter_zscore = spread_zscore_abs <= 5
filtered_index2 = bq.univ.filter(filtered_index, filter_zscore)

req = bql.Request(filtered_index2, {'Z-Spread Outliers':spread_zscore.groupsort()})
res = bq.execute(req)
data = res[0].df()
data

,DATE,Z-Spread Outliers
ID,,
EK016520 Corp,2021-04-08,1.119082
BH786060 Corp,2021-04-08,1.029236
BK877830 Corp,2021-04-08,-0.404529
EJ462516 Corp,2021-04-08,-0.825608
EJ370495 Corp,2021-04-08,-0.918181


#### 2.6.1 Groupxyz Practice: Screening out Outliers

What was my average OAS spread move for financial bonds in the US Corporate (LBUSTRUU Index) for the last 120 days if we remove the top 10% of outliers in our dataset? (Hint: For Sector - use BICS Level 1)

In [ ]:
#Enter answer here
index = bq.univ.members('LBUSTRUU Index')
sprd_chg = bq.data.spread(SPREAD_TYPE='OAS', dates=bq.func.range('-120d','0d'), fill='prev').pct_chg()
sector = bq.data.classification_name('BICS','1')
filter_financials = sector == 'Financials'
filtered_financial_bonds = bq.univ.filter(index, filter_financials)
#getting the top 10% outliers
sprd_chg_zscore = sprd_chg.groupzscore().abs()
zscore_pctl = sprd_chg_zscore.group().cut(10).ungroup()
filter_pct = zscore_pctl <= 9
filtered_bonds = bq.univ.filter(filtered_financial_bonds, filter_pct)
#Retrieve the avg spread move
sprd_chg_avg = sprd_chg.group().avg()

req = bql.Request(filtered_bonds, {'Average Spread Change':sprd_chg_avg})
res = bq.execute(req)
data = res[0].df()
data

<a id='learn4-topic2.7'></a>

### 2.7 Issuer



- From bond level, we can navigate aggregate and manipulate data on the issuing entity
- issuerof()
- parent(type= )

<img src="../../Visualisations/Issuer and Parent.jpg" style="width: 600px;"/>

In [4]:
#Count the number of issuers in Bloomberg Barclays Pan Europe HY Index (LP01TREU Index)
index = bq.univ.members('LP01TREU Index')
issuers = bq.univ.parent(index, type='Ultimate')
#count names
name = bq.data.name()
name_count = name.group().count()

req = bql.Request(issuers, {'Total Securities':name_count})
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS,Total Securities
ID,,
IdentityGroup,None,347


#### 2.7.1 Issuer Practice: Probability of default
- What is the average 1yr probability of default for the ultimate parents of bonds in the Bloomberg Barclays Pan Europe HY Index ('LP01TREU Index')?

In [ ]:
#Please enter answer here
index = bq.univ.members('LP01TREU Index')
ult_parents = bq.univ.parent(index, type='Ultimate')
prob_default = bq.data.probability_of_default(time_horizon='3Y')
avg_pd = prob_default.group().avg()

req = bql.Request(ult_parents, {'Avg Probability of Default':avg_pd})
res = bq.execute(req)
data = res[0].df().head()
data

<a id='learn4-topic2.8'></a>

### 2.8 Value



- introduces data from outside the universe
- can specify a relationship if it exists e.g. mapby=lineage()

<img src="../../Visualisations/BQL Value.jpg" style="width: 600px;"/>

In [5]:
#Getting Issuer Name for a bond
bond = 'AM2044296 Corp'
#Data 1
name = bq.data.name()
#Data 2
issuer_name = name.value(bq.univ.issuerof(), mapby='lineage')

req = bql.Request(bond, {'Issuer Name': issuer_name})
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS:0,Issuer Name
ID,,
AM2044296 Corp,0533805D LN Equity,Virgin Media Secured Finance P


In [6]:
#Getting Yield and Issuer Name for a bond
yld = bq.data.yield_()
name = bq.data.name()
issuer_name = name.value(bq.univ.issuerof(), mapby='lineage')
bond = 'AM2044296 Corp'
#Put our fields in an OrderedDict()
flds = OrderedDict()
flds['Yield (Bond Data)'] = yld
flds['Name (Equity Data)'] = issuer_name

req = bql.Request(bond, flds)
res = bq.execute(req)
tbl = pd.DataFrame({r.name:r.df()[r.name] for r in res})
tbl.head(20)

,Yield (Bond Data),Name (Equity Data)
ID,,
AM2044296 Corp,2.556445,Virgin Media Secured Finance P


In [7]:
#Count the number of USD green bonds issued by issuer since 2019 
all_bonds = bq.univ.bondsuniv('All')
#Set out criteria
criteria_issue_dt = bq.data.issue_dt() >= '2019-01-01' 
criteria_green = bq.data.green_bond_loan_indicator() == 'True'
criteria_crncy = bq.data.crncy() == 'USD'
#Combine our filter criteria
green_criteria = bq.func.and_(criteria_crncy, bq.func.and_(criteria_green, criteria_issue_dt))
green_filter = bq.univ.filter(all_bonds, green_criteria)
#count the bonds in our filtered universe by issuer
security = bq.data.id()
name = bq.data.name()
issuer_name = name.value(bq.univ.issuerof(), mapby='lineage')
count_by_issuer = security.group(issuer_name).count()

req = bql.Request(green_filter, {'Green Bonds by Issuer':count_by_issuer.groupsort()})
res = bq.execute(req)
data = res[0].df().head(30)
data

,ORIG_IDS:0,ORIG_IDS:1,"VALUE(NAME(),ISSUEROF(),MAPBY=MAPBY.LINEAGE)",Green Bonds by Issuer
ID,,,,
United States International De,None,,United States International De,22.0
Credit Agricole Corporate & In,None,,Credit Agricole Corporate & In,12.0
International Finance Corp,None,,International Finance Corp,9.0
International Bank for Reconst,None,,International Bank for Reconst,6.0
Modern Land China Co Ltd,None,,Modern Land China Co Ltd,6.0
Vasakronan AB,None,,Vasakronan AB,6.0
Credit Suisse AG/New York NY,None,,Credit Suisse AG/New York NY,5.0
Industrial & Commercial Bank o,None,,Industrial & Commercial Bank o,5.0
AC Energy Finance Internationa,None,,AC Energy Finance Internationa,4.0


#### 2.8.1 Value Practice: Comparable Bonds
- Using AW8986837 Corp, what are the top 5 most similar bonds using a custom factor?
- Please use the bonds for AMZN US, MSFT US and ORCL US as starting universe
- Custom factor: 0.40 Yield to Worst, 0.60 spread

In [8]:
# Please enter answer here
bond = 'AW8986837 Corp'
y = bq.data.yield_()
sprd = bq.data.spread()
factor = y * 0.40 + sprd * 0.60
bond_factor = factor.value(bq.univ.list(bond))
#find my peer bonds
peer_bonds = bq.univ.bonds(['AMZN US Equity','MSFT US Equity','ORCL US Equity'])
#take the difference between factor for the bonds
peer_factors = factor - bond_factor
peer_factors_abs = peer_factors.abs()
#Let's rank them from lowest to highest
factor_rank = peer_factors_abs.grouprank(order='asc')
filter_rank = factor_rank <= 5
filtered_bonds = bq.univ.filter(peer_bonds, filter_rank)
#retrieve the securities
securityid = bq.data.id()

req = bql.Request(filtered_bonds, securityid)
res = bq.execute(req)
data = res[0].df()
data

,ORIG_IDS,ID()
ID,,
BJ808897 Corp,AMZN US Equity,BJ808897 Corp
AQ133599 Corp,AMZN US Equity,AQ133599 Corp
QZ095291 Corp,MSFT US Equity,QZ095291 Corp
AM332115 Corp,MSFT US Equity,AM332115 Corp
AP914138 Corp,ORCL US Equity,AP914138 Corp


[&uarr; Return to Top](#learn4-top)

<a id='learn4-topic3'></a>

<span style="color:green; font-size:2em"> 3. BQL Advanced Summary </span>



<img src="../../Visualisations/BQL Services.jpg" style="width: 600px;"/>

- This is the beginning: Use BQL to dig deep and conduct in-depth cross-asset financial data analysis
- Regular Updates: Webinars, BQNT Spotlight, BQNT Whatsnew
- Connection to other Bloomberg services, eg Portfolio integration (PORT), order management systems (AIM, TOMS), proprietary data (CDE)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="1.3. FI Advanced - Workbook.ipynb">&larr; Back to the FI Advanced Workbook</a>&emsp;&emsp;
    <a href="#learn4-top">&uarr; Back to Top </a>&emsp;&emsp;
    <a href="../../0 Welcome.ipynb">Back to Home &rarr;</a>
    <br>

</p>